<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_radiomic_and_dosimetric_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Example of radiomic & dosimetric feature calculation using CERR

### Install Octave 5.2 and required packages

In [1]:
%%capture
!apt-get update
!apt-get install gcc g++ gfortran make libblas-dev liblapack-dev libpcre3-dev libarpack2-dev libcurl4-gnutls-dev epstool libfftw3-dev transfig libfltk1.3-dev libfontconfig1-dev libfreetype6-dev libgl2ps-dev libglpk-dev libreadline-dev gnuplot-x11 libgraphicsmagick++1-dev libhdf5-serial-dev openjdk-8-jdk libsndfile1-dev llvm-dev lpr texinfo libgl1-mesa-dev libosmesa6-dev pstoedit portaudio19-dev libqhull-dev libqrupdate-dev libqscintilla2-dev libsuitesparse-dev texlive texlive-generic-recommended libxft-dev zlib1g-dev autoconf automake bison flex gperf gzip icoutils librsvg2-bin libtool perl rsync tar qtbase5-dev qttools5-dev qttools5-dev-tools libqscintilla2-qt5-dev wget git libsundials-dev gnuplot x11-apps
!pip install octave_kernel
!pip install oct2py

In [2]:
%%capture
%%bash
LOCATION=$(curl -s https://api.github.com/repos/cerr/octave-colab/releases/latest \
| grep "tag_name" \
| awk '{print "https://github.com/cerr/octave-colab/archive/" substr($2, 2, length($2)-3) ".zip"}') \
; curl -L -o octavecolab.zip $LOCATION

unzip octavecolab.zip -d octavecolab
releaseDir=$(ls octavecolab)
tar xzvf "octavecolab/${releaseDir}/octavecolab.tar.gz"

In [3]:
import os
os.environ['OCTAVE_EXECUTABLE'] = '/content/octavecolab/bin/octave-cli'
os.environ['PATH'] = '/content/octavecolab/bin:' + os.environ['PATH']

In [4]:
%%capture
%load_ext oct2py.ipython

!apt install octave liboctave-dev
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/image-2.12.0.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/io-2.6.1.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/statistics-1.4.2.tar.gz

!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"


### Download CERR


In [5]:
%%capture
!git clone https://github.com/cerr/CERR.git
%cd /content/CERR
!git checkout octave_dev
%cd ..

### Add Octave packages and CERR to path

In [6]:
%%octave
pkg load image
pkg load io
pkg load statistics

In [7]:
#Add CERR-Octave to path
%%octave
cerrPath = '/content/CERR';
addpath(genpath(cerrPath))

### Read radiomics calculation settings

In [8]:
%%octave
sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/CERR_plans/head_neck_ex1_20may03.mat.bz2';
planC = loadPlanC(sampleData, tempdir);
planC = updatePlanFields(planC);
planC = quality_assure_planC(sampleData, planC);

sampleParam = '/content/CERR/Jupyter_Notebooks/demoParams.json';
paramS = getRadiomicsParamTemplate(sampleParam);

CERR>>  Decompressing head_neck_ex1_20may03.mat.bz2...

ans = 0

CERR>>  Loading head_neck_ex1_20may03.mat.bz2...

CERR>>  Loaded head_neck_ex1_20may03.mat.bz2...

### Run feature calculation

In [9]:
%%octave
indexS = planC{end};
strC = {planC{indexS.structures}.structureName};
strName = paramS.structuresC{1};
structNum = getMatchingIndex(strName,strC,'exact');
scanNum = getStructureAssociatedScan(structNum,planC);
doseNum = 1;
binWidth = 0.01;

structFieldName = ['struct_',repSpaceHyp(strName)];

In [10]:
%%capture
%%octave

%% Compute radiomic features
radiomicsFeatS = calcGlobalRadiomicsFeatures...
(scanNum, structNum, paramS, planC);

%% Compute DVH-based features
[doseV,volV] = getDVH(structNum,doseNum,planC);
[doseBinsV,volHistV] = doseHist(doseV,volV,binWidth);
dvhFeatS.meanDose = calc_meanDose(doseBinsV, volHistV);
dvhFeatS.d10 = calc_Dx(doseBinsV, volHistV, 10, 1);
dvhFeatS.v30 = calc_Vx(doseBinsV, volHistV, 30, 0);
dvhFeatS.MOH55 = calc_MOHx(doseBinsV, volHistV, 55);

### Make computed features available in Python 


In [11]:
from oct2py import octave

%octave_pull radiomicsFeatS dvhFeatS
print(radiomicsFeatS['shapeS'])
print(dvhFeatS)

{'majorAxis': 12.732470146787053, 'minorAxis': 8.91029738209561, 'leastAxis': 4.082517261054258, 'flatness': 0.32063827474077783, 'elongation': 0.6998090142268317, 'max3dDiameter': 10.802777290344238, 'max2dDiameterAxialPlane': 9.492101669311523, 'max2dDiameterSagittalPlane': 6.020797252655029, 'max2dDiameterCoronalPlane': 10.581587791442871, 'surfArea': 78.75934058605056, 'volume': 3.4029999999999943, 'filledVolume': 3.4029999999999943, 'Compactness1': 0.0027468434324124952, 'Compactness2': 0.0026808348319102646, 'spherDisprop': 7.198521632892385, 'sphericity': 0.13891741262965365, 'surfToVolRatio': 23.144090680590857}
{'meanDose': 46.96107504826258, 'd10': 66.865, 'v30': 32.515999999999984, 'MOH55': 55.1462897603486}
